# Mount Google Drive

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
%cd './drive/MyDrive/hasoc/task1'

/content/drive/MyDrive/hasoc/task1


# Installing Libraries

In [ ]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install googletrans==4.0.0-rc1
# !pip install googletrans
!pip install accelerate -U
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 14.7 MB/s eta 0:00:

# Imports and Set Seed

In [ ]:
import random
import torch
import numpy as np

GLOBAL_SEED = 10

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


# Reading Translated Data

In [ ]:
# unique_labels = df['label'].unique()
# label2id = {label: id for id, label in enumerate(unique_labels)}
# id2label = {id: label for id, label in enumerate(unique_labels)}
# num_labels = len(unique_labels)
unique_labels = ['HOF', 'NOT']
label2id = {'HOF': 0, 'NOT': 1}
id2label = {0: 'HOF', 1: 'NOT'}
num_labels = 2

In [ ]:
id2label

{0: 'HOF', 1: 'NOT'}

# Defining Model and Tokenize Dataset

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_checkpoint = "jaimin/Gujarati-Model"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels, label2id=label2id, id2label=id2label, ignore_mismatched_sizes=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at jaimin/Gujarati-Model and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

In [ ]:
# from datasets import load_dataset
# from transformers import DataCollatorForLanguageModeling
# from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# # dataset = load_dataset("your_dataset_name")  # Replace with your dataset name
# # dataset = ("your_dataset_name")
# from datasets import Dataset
# dataset = Dataset.from_pandas(df)
# # processed_dataset
# raw_datasets = dataset.train_test_split(test_size=0.1, seed=GLOBAL_SEED, shuffle=True)
# raw_datasets['validation'] = raw_datasets.pop('test')


# def tokenize_function(examples):
#     # return tokenizer(examples["text"], padding="max_length", truncation=True)
#     return tokenizer(examples["english_translation"], padding="max_length", truncation=True)

# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)#, remove_columns=['text', 'hate_label'])

In [13]:
ls

drive/  sample_data/


In [18]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset, DatasetDict

# Load the dataset using the datasets library
# dataset = load_dataset("your_dataset_name")  # Replace with your dataset name

# Convert the dataset to a pandas DataFrame
# df = dataset.to_pandas()


# Split the dataset into train and test sets while maintaining class balance
# X = df.drop('label', axis=1)  # Replace 'target_column' with your actual target column name
# y = df['label']
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y, random_state=GLOBAL_SEED)

train_df = pd.read_csv('1B_train.csv', index_col = 0)
val_df = pd.read_csv('1B_val.csv', index_col = 0)

# Convert the training and testing sets back to datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Tokenization
# tokenizer = AutoTokenizer.from_pretrained("your_pretrained_model")  # Replace with your tokenizer name
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_val_datasets = val_dataset.map(tokenize_function, batched=True)


tokenized_datasets = DatasetDict({"train": tokenized_train_datasets, "validation": tokenized_val_datasets})

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [19]:
# # Convert the train and validation datasets to pandas DataFrames
# train_df = tokenized_datasets["train"].to_pandas()
# val_df = tokenized_datasets["validation"].to_pandas()

# # Save train and validation DataFrames to CSV files
# train_df.to_csv("sinhala_train_dataset.csv", index=False)
# val_df.to_csv("sinhala_val_dataset.csv", index=False)

In [20]:
tokenized_datasets
# tokenized_train_datasets

DatasetDict({
    train: Dataset({
        features: ['created_at', 'text', 'user_screen_name', 'hate_label', 'english_translation', 'label', 'tweet_id', 'input_ids', 'attention_mask'],
        num_rows: 180
    })
    validation: Dataset({
        features: ['created_at', 'text', 'user_screen_name', 'hate_label', 'english_translation', 'label', 'tweet_id', 'input_ids', 'attention_mask'],
        num_rows: 20
    })
})

In [21]:
# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer,
#     mlm=False  # Set to True if your task is Masked Language Modeling
# )
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Train Model

In [22]:
training_args = TrainingArguments(
    output_dir="./output",
    evaluation_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    learning_rate=2e-5,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=processed_dataset["train"],
    # eval_dataset=processed_dataset["validation"],
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator
)

# trainer.train(resume_from_checkpoint="last-checkpoint")
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.718134
2,No log,0.727333
3,No log,0.754043
4,No log,0.783708
5,No log,0.772554


TrainOutput(global_step=115, training_loss=0.5944064596424932, metrics={'train_runtime': 81.3365, 'train_samples_per_second': 11.065, 'train_steps_per_second': 1.414, 'total_flos': 119220658790400.0, 'train_loss': 0.5944064596424932, 'epoch': 5.0})

In [24]:
predictions = trainer.predict(tokenized_datasets["validation"])
# print(predictions.predictions.shape, predictions.label_ids.shape)
preds = np.argmax(predictions.predictions, axis=-1)

In [25]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Evaluate on validation data
results = trainer.evaluate()

# Calculate metrics
validation_loss = results["eval_loss"]
accuracy = accuracy_score(tokenized_datasets["validation"]["label"], preds)
precision, recall, f1, _ = precision_recall_fscore_support(tokenized_datasets["validation"]["label"], preds, average="weighted")

# Print the metrics
print(f"Validation Loss: {validation_loss}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Validation Loss: 0.7181337475776672
Accuracy: 0.4
Precision: 0.45555555555555555
Recall: 0.4
F1 Score: 0.30109890109890114


In [26]:
test_df = pd.read_csv('gujarati_test_translated.csv', index_col = 0)


from datasets import load_dataset
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset, DatasetDict

# Convert the training and testing sets back to datasets
test_dataset = Dataset.from_pandas(test_df)


tokenized_test_datasets = test_dataset.map(tokenize_function, batched=True)


tokenized_datasets["test"] = tokenized_test_datasets

Map:   0%|          | 0/1196 [00:00<?, ? examples/s]

In [27]:
# Save model predictions on the test dataset
test_predictions = trainer.predict(tokenized_datasets["test"])
test_preds = np.argmax(test_predictions.predictions, axis=-1)

# Save the predictions to a file (e.g., CSV)
# test_predictions_df = pd.DataFrame({"predictions": test_preds})
test_predictions_df = test_df.copy()
test_predictions_df['predictions'] = test_preds
test_predictions_df.to_csv("test_predictions_Gujarati-Model.csv")

In [ ]:
test_predictions_df